In [2]:
!pip install --upgrade --force-reinstall transformers==4.36.2 datasets

^C


In [5]:
!pip install --upgrade transformers peft

  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.15.2-cp311-none-win_amd64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pyarrow-19.0.1-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp311-c

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
botocore 1.29.76 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.4.0 which is incompatible.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires python-dateutil==2.8.2, but you have python-dateutil 2.9.0.post0 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0.2


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 1.7 MB/s eta 0:00:06
   ---- ----------------------------------- 1.0/10.4 MB 1.6 MB/s eta 0:00:06
   ----- ---------------------------------- 1.3/10.4 MB 1.7 MB/s eta 0:00:06
   ------- -------------------------------- 1.8/10.4 MB 1.7 MB/s eta 0:00:05
   -------- ------------------------------- 2.1/10.4 MB 1.8 MB/s eta 0:00:05
   --------- ------------------------------ 2.4/10.4 MB 1.8 MB/s eta 0:00:05
   ----------- ---------------------------- 2.9/10.4 MB 1.7 MB/s eta 0:00:05
   ------------ --------------------------- 3.1/10.4 MB 1.7 MB/s eta 0:00:05
   -------------- ------------------------- 3.7/10.4 MB 1.7 MB/s eta 0:00:04
   --------------- ------------------------ 3.9/10.4 MB 1.8 MB/s eta 0:00:04
   ----------------- ---------------------- 4.5/10.4 MB 1.8 MB/s eta 0:00:04
   ---------

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start(

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [27]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
df = pd.read_csv("cleaned_with_complaint.csv")  # عدّل الاسم لو مختلف
df = df.dropna(subset=["Complaint", "Category", "Subcategory"])

In [29]:
df.drop_duplicates(inplace=True)

In [30]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["Complaint"], padding="max_length", truncation=True, max_length=128)

In [31]:
le_cat = LabelEncoder()
df["cat_label"] = le_cat.fit_transform(df["Category"])

In [32]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df[["Complaint", "cat_label"]])
test_dataset = Dataset.from_pandas(test_df[["Complaint", "cat_label"]])

In [33]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/21572 [00:00<?, ? examples/s]

Map:   0%|          | 0/5394 [00:00<?, ? examples/s]

In [34]:
model_cat = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(le_cat.classes_)).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
import transformers
print(transformers.__version__)

4.51.3


In [41]:

from transformers import TrainingArguments
training_args_cat = TrainingArguments(output_dir="./cat_model",
     num_train_epochs=3,
    per_device_train_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='epoch')


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [16]:
!pip install --upgrade transformers

# كويس

In [15]:
df = pd.read_csv('/content/cleaned_with_complaint.csv')

In [28]:
df.shape

(26966, 4)

In [19]:
df[df.duplicated()].head(100)

,Complaint,Category,Subcategory,Priority
619,destroyed leaving neighborhood area power with...,Electrical Issues,Electrical Faults,Medium
7004,services route unreliable regular lastminute c...,Public Transportation,Delayed or Canceled Services,Medium
7006,bus service route often delayed without notice,Public Transportation,Delayed or Canceled Services,Medium
7007,services route unreliable regular lastminute c...,Public Transportation,Delayed or Canceled Services,Medium
7008,services route unreliable regular lastminute c...,Public Transportation,Delayed or Canceled Services,Medium
...,...,...,...,...
7224,needs urgent attention bus service route often...,Public Transportation,Delayed or Canceled Services,Medium
7228,immediate action required services route unrel...,Public Transportation,Delayed or Canceled Services,Medium
7231,bus service route often delayed without notice,Public Transportation,Delayed or Canceled Services,Medium
7233,problem affecting many people daily services r...,Public Transportation,Delayed or Canceled Services,Medium


In [1]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd